In [84]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.api import qqplot

data_root = "/Users/onkar.pandit/PycharmProjects/dummy/TimeSeriesForecast/data/raw/"

def read_csv(file_path):
    df = pd.read_csv(file_path,index_col=False)
    return df

def get_year_month(df):
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    return df

In [41]:
df = read_csv(os.path.join(data_root,"sensex.csv"))
df = get_year_month(df)

In [42]:
df.tail()

,Date,Open,High,Low,Close,year,month
6718,17-January-2024,71998.93,72484.80,71429.30,71500.76,2024,1
6719,18-January-2024,71018.86,71451.29,70665.50,71186.86,2024,1
6720,19-January-2024,71786.74,71895.64,71542.74,71683.23,2024,1
6721,20-January-2024,72008.30,72026.26,71312.71,71423.65,2024,1
6722,23-January-2024,71868.20,72039.20,70234.55,70370.55,2024,1


In [71]:
df_train, df_test = df[df['year']<2021], df[df['year']>=2021]
print(f"total: {len(df)} \t train: {len(df_train)}, test: {len(df_test)}")

total: 6723 	 train: 5964, test: 759


In [60]:
window_size = 13
X =[]
y = []
for i in range(window_size,len(df_test)):
    X.append(df_test['Close'].iloc[i-window_size:i])
    y.append(df_test['Close'].iloc[i])

In [61]:
xx = np.array(X)
xx[0]

array([47868.98, 48176.8 , 48437.78, 48174.06, 48093.32, 48782.51,
       49269.32, 49517.11, 49492.32, 49584.16, 49034.67, 48564.27,
       49398.29])

In [62]:
yy = np.array(y)
yy.shape

(746,)

In [63]:
def get_prev_close_features(data,window_size):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X.append(data['Close'].iloc[i - window_size:i])
        y.append(data['Close'].iloc[i])
    x = np.array(X)
    y = np.array(y)
    return x,y
x_train,y_train = get_prev_close_features(df_train, window_size=13)
x_test, y_test = get_prev_close_features(df_test, window_size=13)

In [65]:
x_train[0]

array([3260.56, 3225.24, 3264.  , 3252.31, 3264.12, 3288.88, 3362.39,
       3418.32, 3453.16, 3492.26, 3646.73, 3455.88, 3422.35])

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [68]:
X_train_scaled[0]

array([-1.11227954, -1.11544036, -1.11219373, -1.11329893, -1.11236763,
       -1.11031735, -1.1040997 , -1.09939056, -1.09649151, -1.09321319,
       -1.07998851, -1.09649817, -1.09945547])

In [81]:
X,Y = get_prev_close_features(df, window_size=13)
print(X.shape,Y.shape)

(6710, 13) (6710,)


In [78]:
train_sample = x.shape[0] - len(df_test)
x_train,y_train = x[0:train_sample],y[0:train_sample]
x_test,y_test = x[train_sample:],y[train_sample:]

assert x_test.shape[0] == len(df_test)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(5951, 13) (5951,)
(759, 13) (759,)


In [83]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset from the tensors
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader with a batch size
batch_size = 32  # Set your desired batch size here
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example: Iterate through the DataLoader
for batch_X, batch_y in dataloader:
    # Your training code here
    print(batch_X.shape, batch_y.shape)


torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([

In [18]:
mod = ARIMA(df_train['Close'], order=(13, 0, 0))
model_fit = mod.fit()
# itc_df['Close_pred'] = res.predict()
# print(res.summary())

/Users/onkar.pandit/miniconda3/envs/tsa/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/onkar.pandit/miniconda3/envs/tsa/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [31]:
len(df_train)

5964

In [13]:
res

In [24]:
forecast = model_fit.forecast(steps=len(df_test)) 

In [25]:
type(forecast)

pandas.core.series.Series

In [26]:
df_test['pred_values'] = forecast


/var/folders/g6/ylqy09q54v911zyx1bmf687c0000gp/T/ipykernel_17974/4244669927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['pred_values'] = forecast


In [27]:
df_test.head()

,Date,Open,High,Low,Close,year,month,pred_values
5964,1-January-2021,47785.28,47980.36,47771.15,47868.98,2021,1,47773.705939
5965,4-January-2021,48109.17,48220.47,47594.47,48176.80,2021,1,47841.244664
5966,5-January-2021,48037.63,48486.24,47903.38,48437.78,2021,1,47635.722495
5967,6-January-2021,48616.66,48616.66,47864.14,48174.06,2021,1,47971.207785
5968,7-January-2021,48524.36,48558.34,48037.87,48093.32,2021,1,47726.011125


In [28]:
!which python

/Users/onkar.pandit/miniconda3/bin/python


In [29]:
!python

Python 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
>>> 

In [30]:
s = 12

In [85]:
file_path = "/Users/onkar.pandit/Downloads/dummy_data.csv"
data = read_csv(file_path)

In [86]:
data.head()

,language,post_index,commentText,report_count_comment,report_count_post,like_count_comment,like_count_post,label,val
0,Hindi,238566,शायद योगी जी है,0,0,1,1,0,0
1,Hindi,7009,Tingri h to putri tu.. .,0,0,0,0,0,0
2,Hindi,404648,Saale Tu kon sa pagal Nahi h . Teri comment pa...,0,0,0,0,1,0
3,Hindi,5057,girl 😘😘 aaj ke baad msg ki to maar daluggi,0,0,0,0,0,0
4,Hindi,107146,Free fire pubg ka baap ha kutta sala kamina,0,0,0,0,1,0


In [87]:
len(data)

10